In [1]:
import xarray as xr
import fsspec
from google.cloud import storage
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/adamhunter/Documents/school projs/firenet/data/credentials/firenet-99-reader.json'

In [2]:
blob_name = 'ABI-L1b-RadC/2023/341/22/OR_ABI-L1b-RadC-M6C16_G18_s20233412211186_e20233412213572_c20233412214050.nc'

# Use fsspec to create a file system
fs = fsspec.filesystem('gcs', token=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

# Open the file directly into memory with xarray
f = fs.open(f'gcp-public-data-goes-18/{blob_name}')
ds = xr.open_dataset(f, engine='h5netcdf')

In [3]:
# Print the keys of the dataset
print(ds.keys())

KeysView(<xarray.Dataset>
Dimensions:                                           (y: 1500, x: 2500,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       band: 1,
                                                       number_of_harmonization_coefficients: 3,
                                                       num_star_looks: 24)
Coordinates:
    t                                                 datetime64[ns] ...
  * y                                                 (y) float64 0.1282 ... ...
  * x                                                 (x) float64 -0.06997 .....
    y_image                                           float32 ...
    x_image                                           float32 ...
    band_id                                           (band) int8 ...
    band_wavelength                                  

In [4]:
import geopandas as gpd
from shapely.geometry import Polygon

# Get the image bounds
x_bounds = ds['x_image_bounds'].values
y_bounds = ds['y_image_bounds'].values

# Create a polygon from the image bounds
polygon = Polygon([(x_bounds[0], y_bounds[0]), (x_bounds[1], y_bounds[0]), (x_bounds[1], y_bounds[1]), (x_bounds[0], y_bounds[1])])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({
    'geometry': [polygon],
})

# Convert to GeoJSON
geojson = gdf.to_json()

print(geojson)

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[-0.07000000029802322, 0.12824000418186188], [0.07000000029802322, 0.12824000418186188], [0.07000000029802322, 0.04424000158905983], [-0.07000000029802322, 0.04424000158905983], [-0.07000000029802322, 0.12824000418186188]]]}}]}


In [5]:
from google.cloud import storage
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/adamhunter/Documents/school projs/firenet/data/credentials/firenet-99-writer.json'

def upload_blob(bucket_name, geojson, destination_blob_name):
    """Uploads a GeoJSON string to the bucket."""
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(geojson)

    print(
        "GeoJSON uploaded to {}.".format(destination_blob_name)
    )

# Call the function to upload your GeoJSON
upload_blob('prediction_imagery', geojson, 'predictions.geojson')

GeoJSON uploaded to predictions.geojson.
